In [ ]:
!wget https://ai-data.obs.ru-moscow-1.hc.sbercloud.ru/%D0%98%D0%98_%D1%84%D0%BE%D1%82%D0%BE%D0%B1%D0%B0%D0%BD%D0%BA_%D0%B4%D0%BB%D1%8F_%D0%BF%D0%BE%D0%B8%D1%81%D0%BA%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE_%D0%BE%D1%82%D1%80%D1%8F%D0%B4%D0%B0.zip

In [ ]:
%%capture
!unzip ИИ_фотобанк_для_поискового_отряда.zip

In [ ]:
%%capture
!pip install ruclip==0.0.1

In [1]:
import torch
import ruclip

from PIL import Image
import os
from tqdm.notebook import tqdm
from collections import Counter
import shutil
from zipfile import ZipFile

In [2]:
model, processor = ruclip.load("ruclip-vit-large-patch14-336", device="cuda")

In [3]:
paths = ["Итоговый датасет/"]

In [4]:
def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size

In [5]:
results = []
batch_size = 128

with torch.no_grad():
    for path in tqdm(paths):
        data = [os.path.join(path, filename)
                for filename in os.listdir(path)]
        data.sort()
        batches = list(gen_batch(inputs=data, batch_size=batch_size))
        for batch in tqdm(batches):
            pil_images = []
            for idx, path2img in enumerate(batch):
                try:
                    image = Image.open(path2img)
                    pil_images.append(image)
                except:
                    del batch[idx]
            batch_torch = processor(images=pil_images,
                                    return_tensors='pt',
                                    padding=True)["pixel_values"].to("cuda")
            preds = model.encode_image(batch_torch)
            for idx, (path2img, embeddings) in enumerate(zip(batch, preds)):
                results.append([path2img, embeddings])

In [ ]:
results = [(i, j.detach().cpu().numpy()) for i, j in results]

In [26]:
import pandas as pd
from humanize import naturalsize

In [36]:
df = pd.DataFrame([(i, *j) for i, j in results], columns=["path"] + [f"embeddins_{i}" for i in range(768)])

In [37]:
df.head()

In [38]:
df.to_parquet("data.parquet")

In [47]:
df.to_feather("data.feather")

In [46]:
naturalsize(os.path.getsize("data.parquet"))

In [48]:
naturalsize(os.path.getsize("data.feather"))